# Chapter 7. Function Decorators and Closures

## Decorators 101

- Decorators are syntactic sugar.
- Decorators process or replace the passed function or callble, and return it.

Points  
- Decorators can replace the decorated function with different one.
- Decorators are executed immediately when a module is loaded (i.e. import time).

In [1]:
@decorate
def target():
    print('running target()')

# the same
def target():
    print('running target()')

target = decorate(target)

NameError: name 'decorate' is not defined

In [2]:
# A decorator usually replaces a functoin with a different one
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

In [3]:
target()

running inner()


In [4]:
target

<function __main__.deco.<locals>.inner()>

## When Python Executes Decorators

In [6]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
    
if __name__ == '__main__':
    main()

running register(<function f1 at 0x112fc7378>)
running register(<function f2 at 0x112fc7510>)
running main()
registry -> [<function f1 at 0x112fc7378>, <function f2 at 0x112fc7510>]
running f1()
running f2()
running f3()


## Decorator-Enhanced Strategy Pattern

In [7]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 10:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {items.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)

## Closures

In [8]:
# Class-based approach to store previous value
class Averager():
    def __init__(self):
        self.series = []
    
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [9]:
avg = Averager()
avg(10)

10.0

In [10]:
avg(11)

10.5

In [11]:
avg(12)

11.0

In [14]:
# High-order function approach to store previous value
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

In [15]:
avg = make_averager()
avg(10)

10.0

In [16]:
avg(11)

10.5

In [17]:
avg(12)

11.0

In [18]:
avg.__code__.co_varnames

('new_value', 'total')

In [19]:
avg.__code__.co_freevars

('series',)

In [20]:
avg.__closure__

(<cell at 0x112f1c948: list object at 0x112fb0bc8>,)

In [21]:
avg.__closure__[0].cell_contents

[10, 11, 12]

## The nonlocal Declaration

In [23]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

## Implementing a Simple Decorator

In [27]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [28]:
import time

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n - 1)

print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12787887s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000125s] factorial(1) -> 1
[0.00009786s] factorial(2) -> 2
[0.00014038s] factorial(3) -> 6
[0.00017635s] factorial(4) -> 24
[0.00021360s] factorial(5) -> 120
[0.00024432s] factorial(6) -> 720
6! = 720


In [ ]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg from args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r ' % (elapsed, name, arg_str, result))
        return result
    return clocked

## Decorators in the Standard Library

### Memoization with functools.lru_cache

In [29]:
# The very costly recursive way
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

print(fibonacci(6))

[0.00000051s] fibonacci(0) -> 0
[0.00000076s] fibonacci(1) -> 1
[0.00027948s] fibonacci(2) -> 1
[0.00000041s] fibonacci(1) -> 1
[0.00000044s] fibonacci(0) -> 0
[0.00000034s] fibonacci(1) -> 1
[0.00002759s] fibonacci(2) -> 1
[0.00006278s] fibonacci(3) -> 2
[0.00037816s] fibonacci(4) -> 3
[0.00000033s] fibonacci(1) -> 1
[0.00000031s] fibonacci(0) -> 0
[0.00000035s] fibonacci(1) -> 1
[0.00002978s] fibonacci(2) -> 1
[0.00005839s] fibonacci(3) -> 2
[0.00000030s] fibonacci(0) -> 0
[0.00000033s] fibonacci(1) -> 1
[0.00002593s] fibonacci(2) -> 1
[0.00000031s] fibonacci(1) -> 1
[0.00000049s] fibonacci(0) -> 0
[0.00000037s] fibonacci(1) -> 1
[0.00002684s] fibonacci(2) -> 1
[0.00005976s] fibonacci(3) -> 2
[0.00022234s] fibonacci(4) -> 3
[0.00032946s] fibonacci(5) -> 5
[0.00074311s] fibonacci(6) -> 8
8


In [32]:
# Faster implementation using caching
import functools

@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

print(fibonacci(6))

[0.00000068s] fibonacci(0) -> 0
[0.00000108s] fibonacci(1) -> 1
[0.00032321s] fibonacci(2) -> 1
[0.00000170s] fibonacci(3) -> 2
[0.00037890s] fibonacci(4) -> 3
[0.00000117s] fibonacci(5) -> 5
[0.00043285s] fibonacci(6) -> 8
8


### Generic Functions with Single Dispatch

In [38]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return f'<p>{content}</p>'

@htmlize.register(numbers.Integral)
def _(n):
    return f'<pre>{n} (0x{n:x})</pre>'

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n</li>' + inner + '</li>\n</ul>'

## Parameterized Decorators

### A Parameterized Registration Decorator

In [40]:
registry = set()

def register(active=True):
    def decorate(func):
        print('running regiter(acrive=%s)->decorate(%s)' % (active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')
    
@register(active=True)
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')

running regiter(acrive=False)->decorate(<function f1 at 0x112fc7730>)
running regiter(acrive=True)->decorate(<function f2 at 0x1130009d8>)


In [41]:
registry

{<function __main__.f2()>}

In [42]:
# without @
register()(f3)

running regiter(acrive=True)->decorate(<function f3 at 0x112fc7c80>)


<function __main__.f3()>

### The Parameterized Clock Decorator

In [44]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

@clock()
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

[0.12318707s] snooze(0.123) -> None
[0.12405992s] snooze(0.123) -> None
[0.12793088s] snooze(0.123) -> None


In [45]:
import time

@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.12709975242614746s
snooze: 0.12406492233276367s
snooze: 0.12470698356628418s
